---
layout: post
title: API do IBGE
subtitle: Exercícios e Referências
tags: [python, pycharm, jupyter, package, ibge, api]
image: /img/posts/icon_pandas.png
bigimg: /img/posts/big_pandas.png
gh-repo: michelmetran/api_ibge
gh-badge: [follow, star, watch, fork]
comments: true

---

A <a title="Link da API" href="https://servicodados.ibge.gov.br" target="_blank">**_API do IBGE_**</a> possibilita baixar os dados diretamente para o _script_. Para baixar as malhas, ou seja, informações geoespaciais, é possivel baixar os dados em três diferentes formatos:
- **_?formato=application/vnd.geo+json_**, para baixar os dados em GeoJson;
- **_?formato=application/json_**, para baixar os dados em TopoJson;
- **_?formato=image/svg+xml_**, para baixar os dados em SVG;

É possível definir a resolução que, na prática, refere-se ao nível de detalhamento do dado obtido.
- **_?resolucao=0_**, Nenhuma divisão político-administrativa é inserida no interior da malha
- **_?resolucao=1_**, Inclui na malha as macrorregiões. Válido apenas quando a localidade for BR.
- **_?resolucao=2_**, Inclui na malha as Unidades da Federação. Válido apenas quando a localidade for BR ou uma macroregião
- **_?resolucao=3_**, inclui na malha as mesorregiões. Válido apenas quando a localidade for BR, macroregião ou Unidade da Federação
- **_?resolucao=4_**, Inclui na malha as microrregiões. Válido apenas quando a localidade for BR, macroregião, Unidade da Federação ou mesorregião
- **_?resolucao=5_**, inclui na malha os municípios

E a qualidade.
- **_?qualidade=1_**, pior qualidade;
- **_?qualidade=2_**, razoável qualidade;
- **_?qualidade=3_**, boa qualidade;
- **_?qualidade=4_**, melhor qualidade;


{: .alert .alert-danger}
**Aviso:** Esse _post_ tem a finalidade de mostrar os comandos básicos e me deixar com uma "cola" rápida para meu uso cotidiano. Todas os códigos são exemplificativos e podem/devem ser alterados, indicando o nome dos arquivos e diretórios corretamente.

{: .box-note}
**Nota:** É possível acessar esse _post_ em formato <a title="Link do Folium" href="https://github.com/michelmetran/api_ibge/raw/master/docs/api_ibge.pdf" target="_blank">**_pdf_**</a>, diretamente por meio do <a title="Link do Repositório" href="https://github.com/michelmetran/api_ibge" target="_blank">**repositório do GitHub**</a> ou ainda, de maneira interativa, usando o [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/michelmetran/api_ibge/master).

<br>

# Pastas: início do projeto
Inicialmente faz-se necessário criar uma pasta que receberão os dados.

In [104]:
# %load '~/Documents/SourceCode/codes/files/create_folders.py'
def create_folders(path, folders=['data', 'docs', 'maps']):
    """
    :param folders: Name os folders that you want create; E.g.: ['folder1', 'folder2']
    :return: Create directories if not exist
    """
    # Import Packages
    import os
    for folder in folders:
        directory=os.path.join(path, folder)
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
                print('Directory "', directory, '" created!', sep='')
            else:
                print('Directory "', directory, '" already exists...', sep='')
        except OSError:
            print('Error: Creating directory "', directory, '" fail.', sep='')


In [105]:
create_folders('')

Directory "data" already exists...
Directory "docs" already exists...
Directory "maps" already exists...


Com a estrutura de pastas criada, é possivel fazer o download dos arquivos disponiblizados pelo IBGE. Há uma infinidade de dados e ainda, há a <a title="Link da API" href="http://api.sidra.ibge.gov.br" target="_blank">**_API do SIDRA_**</a> que possibilita obter mais dados.

In [106]:
import os
import urllib.request
import shutil

url = 'https://servicodados.ibge.gov.br/api/v2/malhas/35/?resolucao=5&formato=application/vnd.geo+json&qualidade=4'
#url = 'http://servicodados.ibge.gov.br/api/v1/localidades/estados/35/municipios/?formato=application/vnd.geo+json/?resolucao=5'

filename = os.path.join('data', 'divadmin_sp.json')

# Download the file from 'url' and save it locally under 'filename'
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
       shutil.copyfileobj(response, out_file)

Enfrentei problemas com o encoding do arquivo baixado. COm a função abaixo é possivel conferir que o encoding está correto (ascii) para dar continuidade.

In [107]:
# %load '~/Documents/SourceCode/codes/files/predict_encoding.py'
def predict_encoding(file_path, n_lines=30):
    """
    ssss
    :param file_path:
    :param n_lines:
    :return:
    """
    import chardet

    # Open the file as binary data
    with open(file_path, 'rb') as f:
        # Join binary lines for specified number of lines
        rawdata = b''.join([f.readline() for line in range(n_lines)])

    return chardet.detect(rawdata)['encoding']

# file_encoding = predict_encoding('./data.csv')
# print(file_encoding)
# df = pd.read_csv('./data.csv', delimiter=';', encoding=file_encoding)


In [108]:
file_encoding = predict_encoding(filename)
print(file_encoding)

ascii


In [109]:
import folium

# Cria o mapa
webmap = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=8
)

folium.GeoJson(filename, name='Trajetos').add_to(webmap)

webmap

Ou ainda, ao invés de baixar o arquivo, é possivel fazer com o que o mapa seja criado com a leitura dos dados diretamente do site do IBGE. Nessa função o encoding já foi definido, evitando o problema mencionado acima.

In [110]:
import json
import urllib.request
#url = "http://api.openweathermap.org/data/2.5/weather?q=Boras,SE"
webURL = urllib.request.urlopen(url)
data = webURL.read()
encoding = webURL.info().get_content_charset('utf-8')

JSON_object = json.loads(data.decode(encoding))
JSON_object

{'type': 'FeatureCollection',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4674'}},
 'features': [{'type': 'Feature',
   'properties': {'codarea': '3509106',
    'centroide': [-51.986924330008144, -21.801641934836873]},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-51.86, -21.5807],
       [-51.8427, -21.6122],
       [-51.864, -21.6705],
       [-51.8831, -21.6871],
       [-51.8993, -21.7206],
       [-51.9007, -21.7359],
       [-51.8878, -21.7635],
       [-51.9, -21.78],
       [-51.9134, -21.8201],
       [-51.9337, -21.8278],
       [-51.9473, -21.8563],
       [-51.9634, -21.9064],
       [-52.0028, -21.9123],
       [-52.0249, -21.9391],
       [-52.023, -21.9486],
       [-52.032, -21.9878],
       [-52.0275, -22.0037],
       [-52.0384, -22.0269],
       [-52.0303, -22.048],
       [-52.0557, -22.0333],
       [-52.0674, -22.0088],
       [-52.0883, -21.9874],
       [-52.1105, -21.9811],
       [-52.1439, -21.9984],
       [-52.12

In [111]:
import folium

# Cria o mapa
webmap = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=8
)

folium.GeoJson(JSON_object, name='Trajetos').add_to(webmap)

webmap

Uma vez com o mapa na mão, de qualquer que seja o meio que foi obtido, é possivel analisar a "tabela de atributos".
Lá descobrimos que existe o par de coordenadas que define o centroide e, ainda, o 'codarea' que tem o código do IBGE do município.

Missão é:
Renomear o campo.
Dar joins entre jsons
plotar colorido!

In [112]:
#url = 'http://portalgeo.seade.gov.br/wp-content/uploads/2019/03/LimiteMunicipal_IGC.zip'
# Se lê certo, no notepad aparecer UTF-8 BOM WO

___
<br>

# Exportando o _Juptyter Notebook_ para outros formatos
O arquivo _.ipynb_ pode ser exportado em formatos diversos. Abaixo carrego uma função que escrevi para facilitar o processo de exportação do arquivo em diferentes locais do PC para, posteriormente, atualizar os repositórios contidos no <a title="Link do GitHub" href="https://github.com/michelmetran" target="_blank">_GitHub_</a>.

In [33]:
# %load '~/Documents/SourceCode/codes/files/export_jupyter.py'
def export_jupyter(path, extensions=['html', 'markdown', 'latex', 'pdf', 'python'], today=True):
    """
    Export .ipynb file to others formats
    :return: File in other formats
    """
    # Import Packages
    import os
    import datetime

    # Data
    timestamp = datetime.datetime.now()
    srt_today = (str(timestamp.year) + '-' +
                 str(f"{timestamp.month:02d}") + '-' +
                 str(f"{timestamp.day:02d}"))

    # Extensions
    for extension in extensions:
        if today==True:
            os.system('jupyter nbconvert --to {} {} --output {}'.
                      format(extension, get_jupyternotebook_name(),
                             os.path.join(path, srt_today+'-'+get_jupyternotebook_name().split('.')[0])))
            print('Arquivo {} exportado corretamente para o formato {} usando prefixo da data.'.
                  format(get_jupyternotebook_name(), extension))

        else:
            os.system('jupyter nbconvert --to {} {} --output {}'.
                      format(extension, get_jupyternotebook_name(),
                             os.path.join(path, get_jupyternotebook_name().split('.')[0])))
            print('Arquivo {} exportado corretamente para o formato {} sem usar prefixo da data.'.
                  format(get_jupyternotebook_name(), extension))


In [34]:
# %load '~/Documents/SourceCode/codes/files/get_jupyternotebook_name.py'
def get_jupyternotebook_name():
    """
    Returns the name of the current notebook as a string
    From https://mail.scipy.org/pipermail/ipython-dev/2014-June/014096.html
    :return: Returns the name of the current notebook as a string
    """
    # Import Packages
    from IPython.core.display import Javascript
    from IPython.display import display

    display(Javascript('IPython.notebook.kernel.execute("theNotebook = " + \
    "\'"+IPython.notebook.notebook_name+"\'");'))

    # Result
    return theNotebook


Com as funções para exportar o _Jupyter Notebook_ e para obter o nome do arquivo _.ipynb_ carregadas, basta exportar o arquivo, inicialmente para a pasta _docs_ dentro do projeto e também, visando atualizar os _posts_ do site, para a respectiva pasta.

In [35]:
export_jupyter('docs',['pdf'], False)
export_jupyter('/home/michel/Documents/SourceCode/michelmetran.github.io/_posts', ['markdown'], True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo api_ibge.ipynb exportado corretamente para o formato pdf sem usar prefixo da data.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo api_ibge.ipynb exportado corretamente para o formato markdown usando prefixo da data.


<br>

# Atualizando Repositórios
Após as exportações dos arquivos nos formatos necessários, basta atualizar o repositório diretamente pelo  _Jupyter Notebook_.
Abaixo é atualizado o repositório desse projeto específico, bem como a derivação desse projeto no <a title="Link do Folium" href="https://michelmetran.github.io/" target="_blank">**_site_**</a>.

In [36]:
%run '~/Documents/SourceCode/codes/git/update_github.py'

In [37]:
git_full('/home/michel/Documents/SourceCode/api_ibge', '.', 'Atualizando')
git_full('/home/michel/Documents/SourceCode/michelmetran.github.io', '.', 'Atualizando')

b'' b''
b'[master 098a00f] Atualizando\n 2 files changed, 110 insertions(+), 470 deletions(-)\n delete mode 100644 .ipynb_checkpoints/api_ibge-checkpoint.ipynb\n' b''
b'' b'To github.com:michelmetran/api_ibge.git\n   0684fb4..098a00f  master -> master\n'
Done!!
b'' b''
b"On branch master\nYour branch is up to date with 'origin/master'.\n\nnothing to commit, working tree clean\n" b''
b'' b'Everything up-to-date\n'
Done!!


# _Requirements_
Abaixo é criado o arquivo _requirements.txt_ na raiz do projeto para possibilitar o correto funcionamento do _Jupyter Notebook_ no <a title="Link do My Binder" href="https://mybinder.org/" target="_blank">**_My Binder_**</a>. Após a criação do arquivo, sugere-se a edição manual, visando manter apenas os _packages_ realmente essenciais, listados com o comando _import_ no início do _script_.

In [ ]:
#pip freeze > requirements.txt